In [1]:
import torch.nn.functional as F
from torch import nn, einsum
import torch
from einops import rearrange
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import math
from inspect import isfunction
from functools import partial

%matplotlib inline


/usr/local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## diffusion-modelとは？

- 単純な分布から得たノイズをデータサンプルに変換するモデル
- 純粋なノイズからNNが徐々にノイズ除去の方法を学習していく

以下の二つのプロセスから構成される

### Forward diffusion process
- データにガウス分布から生成したノイズを加算していき、十分純粋なノイズとなるまでTステップ繰り返す
    - DDPMでは1000くらい
    - 十分Tが大きく、また「a well behaved schedule for adding noise at each time step」であればisotropic Gaussian distributionと見なせるらしい
        - https://math.stackexchange.com/questions/1991961/gaussian-distribution-is-isotropic

### Learned reverse denoising diffusion process
- NNが純粋なノイズから元の画像になるまでノイズ除去できるよう学習する

In [3]:
# NN実装用のヘルパー関数

def exists(x):
    return x is not None

def default(val, d):
    if exists(val):
        return val
    return d() if isfunction(d) else d

class Residual(nn.Module):
    def __init__(self, fn):
        super().__init__()
        self.fn = fn

    def forward(self, x, *args, **kwargs):
        """
        docstring
        """
        return self.fn(x, *args, **kwargs) + x



def Upsample(dim):
    """
    アップサンプル用の関数
    """
    return nn.ConvTranspose2d(dim, dim, 4,2,1)

def Downsample(dim):
    """
    ダウンサンプリング
    """
    return nn.Conv2d(dim, dim, 4,2,1)


## tの埋め込み

- transformerと同じくPositional Embeddingsを行う

In [6]:
# tのpositionalEmbeddingクラス

class SinusoidalPositionEmbeddings(nn.Module):
    def __init__(self, dim):
        """
        dimは埋め込み先次元
        """
        super().__init__()
        self.dim = dim
    
    def forward(self, time:torch.Tensor):
        """
        tの埋め込み
        """
        device = time.device
        half_dim = self.dim // 2

        embeddings = math.log(10000) / (half_dim-1)
        embeddings = torch.exp(torch.arange(
            half_dim, device=device
        ) * -embeddings)
        embeddings = time[:, None] * embeddings[None, :]
        embeddings = torch.cat((embeddings.sin(), embeddings.cos()), dim=-1)
        return embeddings


In [ ]:
class Block(nn.Module):
    """
    docstring
    """
    def __init__(self,dim, dim_out, groups=8):
        super().__init__()
        self.proj = nn.Conv2d(dim, dim_out, 3, padding=1)
        self.norm = nn.GroupNorm(groups, dim_out)
        self.act = nn.SiLU()
    
    def forward(self, c, scale_shift=None):
        """
        docstring
        """
        x = self.proj(x)
        x = self.norm(x)

        if exists(scale_shift):
            scale, shift = scale_shift
            x = x*(scale + 1) + shift

        
        x = self.act(x)
        return x